# Numpy Broadcasting 

## Overview 

Broadcasting is a valuable part of the power that NumPy provides. However, there's no looking past the fact that broadcasting can be conceptually difficult to digest. This information can be helpful and very powerful, but we also suggest moving on to take a look at some of the label-based corners of the Python ecosystem, namely Pandas and Xarray for the ways that they make some of these concepts simpler or easier to use for real-world data.

    1. An introduction to broadcasting 
    2. Avoiding loops with vectorization 
    

## Imports 

In [1]:
import numpy as np 

## Using broadcasting to implicitly loop over data

### What is broadcasting?

Broadcasting is a useful NumPy tool that allows us to perform operations between arrays with different shapes, provided that they are ompatible with each other in certain ways. 

To start we create an array below and add 5 to it:


In [2]:
a = np.array([10, 20, 30, 40])
a + 5

array([15, 25, 35, 45])

In [3]:
a

array([10, 20, 30, 40])

This works even though 5 is not an array; it works like we would expect, adding 5 to each of the elements in a. This also works if 5 is an array:

In [4]:
b =np.array([5])
a + b


array([15, 25, 35, 45])

This takes the single element in b and adds it to each of the elements in a. this wont work for just any b, though; For instance, the following:

In [5]:
b = np.array([5, 6, 7])
a + b

ValueError: operands could not be broadcast together with shapes (4,) (3,) 

Wont work, it does work if a and b are the same shape:

In [6]:
b = np.array([5, 5, 10, 10])
a + b

array([15, 25, 40, 50])

What if what we really want is pairwise addition of a, b? Without broadcasting, we could accomplish this by looping:

In [16]:
b =np.array([1, 2, 3, 4, 5])
b

array([1, 2, 3, 4, 5])

In [11]:
result = np.empty((5,4), dtype=np.int32) ##creates empty array to store values 
for row, valb in enumerate(b):
    for col, vala in enumerate(a):
        result[row, col] = vala + valb
result

array([[11, 21, 31, 41],
       [12, 22, 32, 42],
       [13, 23, 33, 43],
       [14, 24, 34, 44],
       [15, 25, 35, 45]])

Quick not on "enumerate:" Often, when dealing with iterators, we also need to keep count of iterations. Python eases the programmers' task by providing built-in function enumerate() for this task. Enumerate() method adds a counter to an iterable and returns it in a form of enumerating object. This enumerated object can then be used directly for loops or converted into a list of tuples using the list() method.

We can also do this by manually repeating the arrays to the proper shape for the result, using np.tile. This avoids the need to manually loop:

In [15]:
aa = np.tile(a, (5, 1))
aa

array([[10, 20, 30, 40],
       [10, 20, 30, 40],
       [10, 20, 30, 40],
       [10, 20, 30, 40],
       [10, 20, 30, 40]])

In [17]:
# Turn b into a column array, then tile it 
bb = np.tile(b.reshape(5, 1), (1, 4))
bb

array([[1, 1, 1, 1],
       [2, 2, 2, 2],
       [3, 3, 3, 3],
       [4, 4, 4, 4],
       [5, 5, 5, 5]])

In [18]:
aa + bb

array([[11, 21, 31, 41],
       [12, 22, 32, 42],
       [13, 23, 33, 43],
       [14, 24, 34, 44],
       [15, 25, 35, 45]])

## Giving NumPy room for broadcasting 

We can also do this using broadcasting, which is where NumPy implicitly repeats the array without using additional memory. With broadcasting, NumPy takes care of repeating for you, provided dimensions are "compatible". This works as:

1. Check the number of dimensions of the arrays, if they are different, prepend size one dimensions
2. Check if each of the dimensions are compatible: 
either the same size or one of them is 1.

In [19]:
a.shape

(4,)

In [20]:
b.shape

(5,)

Right now they have the same number of dimensions, 1, but that dimension is incompatible. We can solve this by appending a dimension using np.newaxis when indexing:

In [24]:
bb = b[:, np.newaxis]
bb.shape

(5, 1)

In [22]:
a + bb

array([[11, 21, 31, 41],
       [12, 22, 32, 42],
       [13, 23, 33, 43],
       [14, 24, 34, 44],
       [15, 25, 35, 45]])

In [25]:
(a + bb).shape

(5, 4)

This can be written more directly in one line:

In [26]:
a + b[:, np.newaxis]

array([[11, 21, 31, 41],
       [12, 22, 32, 42],
       [13, 23, 33, 43],
       [14, 24, 34, 44],
       [15, 25, 35, 45]])

## Extending to higher dimensions 

This also works for higher dimensions. x, y, and z are here different dimensions, and we can broadcast to perform x^2 + y^2 + z^2

In [29]:
x = np.array([1, 2])
y = np.array([3, 4, 5])
z = np.array([6, 7, 8, 9])
x

array([1, 2])

In [30]:
y

array([3, 4, 5])

In [31]:
z

array([6, 7, 8, 9])

First, lets extend x (and square it) by one dimension, onto which we can broadcast the vector y **2 

In [34]:
d_2d = x[:, np.newaxis] ** 2 + y**2

In [35]:
d_2d

array([[10, 17, 26],
       [13, 20, 29]])

In [36]:
d_2d.shape

(2, 3)

And then further extend this new 2-D array by one more dimension before using broadcasting to add z ** 2 across all other dimensions 


In [37]:
d_3d = d_2d[..., np.newaxis] + z**2

In [38]:
d_3d

array([[[ 46,  59,  74,  91],
        [ 53,  66,  81,  98],
        [ 62,  75,  90, 107]],

       [[ 49,  62,  77,  94],
        [ 56,  69,  84, 101],
        [ 65,  78,  93, 110]]])

In [39]:
d_3d.shape

(2, 3, 4)

Or in one line:

In [40]:
h = x[:, np.newaxis, np.newaxis] ** 2 + y[np.newaxis, :, np.newaxis] ** 2 + z**2

In [41]:
h

array([[[ 46,  59,  74,  91],
        [ 53,  66,  81,  98],
        [ 62,  75,  90, 107]],

       [[ 49,  62,  77,  94],
        [ 56,  69,  84, 101],
        [ 65,  78,  93, 110]]])

In [42]:
h.shape

(2, 3, 4)

And we can confirm that the results here are identical 

In [43]:
np.all(h == d_3d)

True

Broadcasting is often useful when you want to do calculations with coordinate values, which are often given as 1-D arrays corresponding to positions along a particular array dimension. For example, taking range and azimuth values for radar data (1-D separable polar coordinates) and converting to x,y pairs relative to the radar location.

Given the 3-D temperature field and 1-D pressure coordinates below, let’s calculate . We will need to use broadcasting to make the arrays compatible!

In [44]:
pressure = np.array([1000, 850, 500, 300])
temps = np.linspace(20, 30, 24).reshape(4, 3, 2)
pressure.shape, temps.shape

((4,), (4, 3, 2))

In [45]:
pressure

array([1000,  850,  500,  300])

In [47]:
temps

array([[[20.        , 20.43478261],
        [20.86956522, 21.30434783],
        [21.73913043, 22.17391304]],

       [[22.60869565, 23.04347826],
        [23.47826087, 23.91304348],
        [24.34782609, 24.7826087 ]],

       [[25.2173913 , 25.65217391],
        [26.08695652, 26.52173913],
        [26.95652174, 27.39130435]],

       [[27.82608696, 28.26086957],
        [28.69565217, 29.13043478],
        [29.56521739, 30.        ]]])

In [50]:
pressure[:, np.newaxis, np.newaxis].shape

(4, 1, 1)

In [53]:
pressure[:, np.newaxis, np.newaxis]

array([[[1000]],

       [[ 850]],

       [[ 500]],

       [[ 300]]])

In [49]:
temps * np.exp(pressure[:, np.newaxis, np.newaxis] / 1000)

array([[[54.36563657, 55.54749823],
        [56.7293599 , 57.91122156],
        [59.09308323, 60.27494489]],

       [[52.89636361, 53.91360137],
        [54.93083913, 55.94807689],
        [56.96531466, 57.98255242]],

       [[41.57644944, 42.29328477],
        [43.01012011, 43.72695544],
        [44.44379078, 45.16062611]],

       [[37.56128856, 38.14818369],
        [38.73507883, 39.32197396],
        [39.90886909, 40.49576423]]])

# Vectorize calculations to avoid explicit loops

When working with arrays of data, loops over the individual array elements is a fact of life. However, for improved runtime performance, it is important to avoid performing these loops in Python as much as possible, and let NumPy handle the looping for you. Avoiding these loops frequently, but not always, results in shorter and clearer code as well.

# Look ahead/behind

One common pattern for vectorizing is in converting loops that work over the current point as well as the previous and/or net point. this comes up when doing finite-difference calculations, e.g. approximating derivatives, 


In [56]:
a = np.linspace(0, 20, 6)
a

array([ 0.,  4.,  8., 12., 16., 20.])

We can calculate the forward difference for this array with a manual loop as:

In [59]:
d = np.zeros(a.size - 1)
for i in range(len(a) - 1):
    d[i] = a[i + 1] - a[i]
d

array([4., 4., 4., 4., 4.])

It would be nice to express this calculation without a loop, if possible. To see how to go about this, let’s consider the values that are involved in calculating d[i], a[i+1] and a[i]. The values over the loop iterations are:

i a[i+1] a[i]
0 4      0   
1 8      4
2 12     8
3 16     12
4 20     16


We can express the series of values for a[i+1] then as 


In [60]:
a[1:]

array([ 4.,  8., 12., 16., 20.])

In [69]:
a[:-1]

array([ 0.,  4.,  8., 12., 16.])

In [63]:
a[1:] - a[:-1]

array([4., 4., 4., 4., 4.])

It should be noted that using slices in this way returns only a view on the original array. This means not only can you use the slices to modify the original data (even accidentally), but that this is also a quick operation that does not involve a copy and does not bloat memory usage.

## 2nd Derivative
A finite difference estimate of the 2nd derivative is given by:

(we’re ignoring  here)

Let’s write some vectorized code to calculate this finite difference for a (using slices.) What values should we be expecting to get for the 2nd derivative?

In [64]:
2 * a[1:-1] - a[:-2] - a[2:]

array([0., 0., 0., 0.])

## Blocking 

Another application where vectorization comes into play to make operations more efficient is when operating on blocks of data. Let's start by creating some temperature data (rounding to make it easier to see/recognize the values)

In [70]:
temps = np.round(20 + np.random.randn(10) * 5, 1)
temps

array([17.1, 23.3, 20.9, 20.7, 20.2, 14.4, 28.1, 24.6, 21.3, 16.3])

Let’s start by writing a loop to take a 3-point running mean of the data. We’ll do this by iterating over all points in the array and average the 3 points centered on that point. We’ll simplify the problem by avoiding dealing with the cases at the edges of the array.

In [71]:
avg = np.zeros_like(temps)
for i in range(1, len(temps) - 1):
    sub = temps[i - 1 : i + 2]
    avg[i] = sub.mean()

In [72]:
avg

array([ 0.        , 20.43333333, 21.63333333, 20.6       , 18.43333333,
       20.9       , 22.36666667, 24.66666667, 20.73333333,  0.        ])

As with the case of doing finite differences, we can express this using slices of the original array:

In [73]:
# i - 1            i          i + 1
(temps[:-2] + temps[1:-1] + temps[2:]) / 3

array([20.43333333, 21.63333333, 20.6       , 18.43333333, 20.9       ,
       22.36666667, 24.66666667, 20.73333333])

Another option to solve this is not using slicing but by using a powerful NumPy tool: as_strided. This tool can result in some odd behavior, so take care when using–the trade-off is that this can be used to do some powerful operations. What we’re doing here is altering how NumPy is interpreting the values in the memory that underpins the array. So for this array:

In [74]:
temps


array([17.1, 23.3, 20.9, 20.7, 20.2, 14.4, 28.1, 24.6, 21.3, 16.3])

we can create a view of the array with a new, bigger shape, with rows made up of overlapping values. We do this by specifying a new shape of 8x3, one row for each of the length 3 blocks we can fit in the original 1-D array of data. We then use the strides argument to control how NumPy walks between items in each dimension. The last item in the strides tuple is just as normal–it says that the number of bytes to walk between items is just the size of an item. (Increasing this would skip items.) The first item says that when we go to a new, in this case row, only advance the size of a single item. This is what gives us overlapping rows.

In [75]:
block_size = 3
new_shape = (len(temps) - block_size + 1, block_size)
bytes_per_item = temps.dtype.itemsize
temps_strided = np.lib.stride_tricks.as_strided(
    temps, shape=new_shape, strides=(bytes_per_item, bytes_per_item)
)
temps_strided

array([[17.1, 23.3, 20.9],
       [23.3, 20.9, 20.7],
       [20.9, 20.7, 20.2],
       [20.7, 20.2, 14.4],
       [20.2, 14.4, 28.1],
       [14.4, 28.1, 24.6],
       [28.1, 24.6, 21.3],
       [24.6, 21.3, 16.3]])

Now that we have this view of the array with the rows representing overlapping blocks, we can operate across the rows with mean and the axis=-1 argument to get our running average:

In [76]:
temps_strided.mean(axis=-1)

array([20.43333333, 21.63333333, 20.6       , 18.43333333, 20.9       ,
       22.36666667, 24.66666667, 20.73333333])

It should be noted that there are no copies going on here, so if we change a value at a single indexed location, the change actually shows up in multiple locations:

In [77]:
temps_strided[0, 2] = 2000
temps_strided

array([[  17.1,   23.3, 2000. ],
       [  23.3, 2000. ,   20.7],
       [2000. ,   20.7,   20.2],
       [  20.7,   20.2,   14.4],
       [  20.2,   14.4,   28.1],
       [  14.4,   28.1,   24.6],
       [  28.1,   24.6,   21.3],
       [  24.6,   21.3,   16.3]])

# Finding the difference between min and max

Another operation that crops up when slicing and dicing data is trying to identify a set of indexes, along a particular axis, within a larger multidimensional array. For instance, say we have a 3-D array of temperatures and want to identify the location of the -10 degree C isotherm within each column.

In [78]:
pressure = np.linspace(1000, 100, 25)
temps = np.random.randn(25, 30, 40) * 3 + np.linspace(25, -100, 25).reshape(-1, 1, 1)

In [79]:
pressure

array([1000. ,  962.5,  925. ,  887.5,  850. ,  812.5,  775. ,  737.5,
        700. ,  662.5,  625. ,  587.5,  550. ,  512.5,  475. ,  437.5,
        400. ,  362.5,  325. ,  287.5,  250. ,  212.5,  175. ,  137.5,
        100. ])

In [81]:
temps.shape

(25, 30, 40)

NumPy has the function argmin() which returns the index of the minimum value. We can use this to find the minimum absolute difference between the value and -10: 

In [82]:
# Using axis=0 to tell it to operate along the pressure dimension
inds = np.argmin(np.abs(temps - -10), axis=0)
inds

array([[4, 6, 8, ..., 7, 5, 7],
       [7, 7, 6, ..., 7, 6, 7],
       [6, 7, 7, ..., 7, 7, 7],
       ...,
       [6, 6, 7, ..., 7, 6, 7],
       [6, 7, 6, ..., 7, 7, 6],
       [6, 6, 7, ..., 7, 7, 6]], dtype=int64)

In [83]:
inds.shape

(30, 40)

Great! We have an array representing the index of the point closest to  in each column of data. We could use this to look up into our pressure coordinates to find the pressure level for each column:

In [84]:
pressure[inds]

array([[850. , 775. , 700. , ..., 737.5, 812.5, 737.5],
       [737.5, 737.5, 775. , ..., 737.5, 775. , 737.5],
       [775. , 737.5, 737.5, ..., 737.5, 737.5, 737.5],
       ...,
       [775. , 775. , 737.5, ..., 737.5, 775. , 737.5],
       [775. , 737.5, 775. , ..., 737.5, 737.5, 775. ],
       [775. , 775. , 737.5, ..., 737.5, 737.5, 775. ]])

How about using that to find the actual temperature value that was closest?

In [85]:
temps[inds, :, :].shape

(30, 40, 30, 40)

Unfortunately, this replaced the pressure dimension (size 25) with the shape of our index array (30 x 40), giving us a 30 x 40 x 30 x 40 array (imagine what would have happened with real data!). One solution here would be to loop:



In [86]:
output = np.empty(inds.shape, dtype=temps.dtype)
for (i, j), val in np.ndenumerate(inds):
    output[i, j] = temps[val, i, j]
output

array([[ -3.77489983, -10.32143841, -12.39903608, ..., -13.06208565,
         -8.17403441, -10.55024164],
       [-14.7169672 ,  -7.56327819,  -6.77939967, ...,  -7.60838438,
         -9.38068418,  -8.79172211],
       [-10.31205858, -10.3959255 , -12.90149417, ...,  -9.17673021,
        -10.80894527, -10.38503076],
       ...,
       [-11.33008063,  -7.32639657, -10.32330325, ...,  -9.8387453 ,
         -9.00235153,  -7.75446749],
       [ -7.94184741,  -8.3525437 ,  -8.25572994, ..., -11.68654102,
        -13.39372387,  -8.78970146],
       [ -8.55217109,  -7.52321101,  -1.88087158, ..., -11.41648748,
         -8.9473684 ,  -6.2892915 ]])

Of course, what we really want to do is avoid the explicit loop. Let’s temporarily simplify the problem to a single dimension. If we have a 1-D array, we can pass a 1-D array of indices (a full) range, and get back the same as the original data array:

In [87]:
pressure[np.arange(pressure.size)]

array([1000. ,  962.5,  925. ,  887.5,  850. ,  812.5,  775. ,  737.5,
        700. ,  662.5,  625. ,  587.5,  550. ,  512.5,  475. ,  437.5,
        400. ,  362.5,  325. ,  287.5,  250. ,  212.5,  175. ,  137.5,
        100. ])

In [88]:
np.all(pressure[np.arange(pressure.size)] == pressure)

True

We can use this to select all the indices on the other dimensions of our temperature array. We will also need to use the magic of broadcasting to combine arrays of indices across dimensions.

Now let’s consider a vectorized solution:

In [89]:
y_inds = np.arange(temps.shape[1])[:, np.newaxis]
x_inds = np.arange(temps.shape[2])
temps[inds, y_inds, x_inds]

array([[ -3.77489983, -10.32143841, -12.39903608, ..., -13.06208565,
         -8.17403441, -10.55024164],
       [-14.7169672 ,  -7.56327819,  -6.77939967, ...,  -7.60838438,
         -9.38068418,  -8.79172211],
       [-10.31205858, -10.3959255 , -12.90149417, ...,  -9.17673021,
        -10.80894527, -10.38503076],
       ...,
       [-11.33008063,  -7.32639657, -10.32330325, ...,  -9.8387453 ,
         -9.00235153,  -7.75446749],
       [ -7.94184741,  -8.3525437 ,  -8.25572994, ..., -11.68654102,
        -13.39372387,  -8.78970146],
       [ -8.55217109,  -7.52321101,  -1.88087158, ..., -11.41648748,
         -8.9473684 ,  -6.2892915 ]])

Let’s say we want to find the relative humidity at the -10 C isotherm

In [90]:
np.all(output == temps[inds, y_inds, x_inds])

True

here [https://numpy.org/doc/stable/user/basics.broadcasting.html]